In [ ]:
%matplotlib inline

import sqlite3

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Prepare data

## Master categories from R

```R
load("master.categories.dat")
install.packages("jsonlite")
library(jsonlite)
toJSON(master.categories, pretty = TRUE)
>
```

```json
{
  "Housing": ["Home", "Home Improvement", "Home Insurance", "Home Services", "Home Supplies", "Lawn & Garden", "Mortgage & Rent"],
  "Food": ["Fast Food", "Food & Dining", "Groceries", "Restaurants"],
  "Bills": ["Bills & Utilities", "Home Phone", "Internet", "Mobile Phone", "Student Loan", "Television", "Utilities", "Web Hosting"],
  "Health": ["Dentist", "Doctor", "Eyecare", "Gym", "Hair", "Health & Fitness", "Health Insurance", "Personal Care", "Pharmacy"],
  "Travel": ["Air Travel", "Hotel", "Travel", "Vacation"],
  "Transport": ["Auto & Transport", "Auto Insurance", "Auto Payment", "Gas & Fuel", "Parking", "Public Transportation", "Registration", "Rental Car & Taxi", "Service & Parts"],
  "Entertainment": ["Amusement", "Alcohol & Bars", "Arts", "Coffee Shops", "Entertainment", "Movies & DVDs", "Music", "Newspapers & Magazines", "Spa & Massage", "Sports"],
  "Stuff": ["Books", "Books & Supplies", "Clothing", "Electronics & Software", "Furnishings", "Hobbies", "Office Supplies", "Pet Food & Supplies", "Printing", "Shopping", "Sporting Goods"],
  "Other": ["Advertising", "ATM Fee", "Bank Fee", "Business Services", "Charity", "Check", "Education", "Federal Tax", "Fees & Charges", "Finance Charge", "Financial", "Gift", "Gifts & Donations", "Late Fee", "Local Tax", "Moving Expenses", "Personal Development", "Pets", "Service Fee", "Shipping", "State Tax", "Taxes", "Trade Commissions", "Uncategorized", "Wedding", "Work Reimbursable"],
  "Ignore": ["Cash & ATM", "Credit Card Payment", "Hide from Budgets & Trends", "Investments", "Kids", "Savings", "Transfer", "Transfer for Cash Spending", "Transfer to Savings"],
  "Income": ["Bonus", "Income", "Interest Income", "Paycheck", "Rental"]
}
```

In [ ]:
mc = {
  "Housing": ["Home", "Home Improvement", "Home Insurance", "Home Services", "Home Supplies", "Lawn & Garden", "Mortgage & Rent"],
  "Food": ["Fast Food", "Food & Dining", "Groceries", "Restaurants"],
  "Bills": ["Bills & Utilities", "Home Phone", "Internet", "Mobile Phone", "Student Loan", "Television", "Utilities", "Web Hosting"],
  "Health": ["Dentist", "Doctor", "Eyecare", "Gym", "Hair", "Health & Fitness", "Health Insurance", "Personal Care", "Pharmacy"],
  "Travel": ["Air Travel", "Hotel", "Travel", "Vacation"],
  "Transport": ["Auto & Transport", "Auto Insurance", "Auto Payment", "Gas & Fuel", "Parking", "Public Transportation", "Registration", "Rental Car & Taxi", "Service & Parts"],
  "Entertainment": ["Amusement", "Alcohol & Bars", "Arts", "Coffee Shops", "Entertainment", "Movies & DVDs", "Music", "Newspapers & Magazines", "Spa & Massage", "Sports"],
  "Stuff": ["Books", "Books & Supplies", "Clothing", "Electronics & Software", "Furnishings", "Hobbies", "Office Supplies", "Pet Food & Supplies", "Printing", "Shopping", "Sporting Goods"],
  "Other": ["Advertising", "ATM Fee", "Bank Fee", "Business Services", "Charity", "Check", "Education", "Federal Tax", "Fees & Charges", "Finance Charge", "Financial", "Gift", "Gifts & Donations", "Late Fee", "Local Tax", "Moving Expenses", "Personal Development", "Pets", "Service Fee", "Shipping", "State Tax", "Taxes", "Trade Commissions", "Uncategorized", "Wedding", "Work Reimbursable"],
  "Ignore": ["Cash & ATM", "Credit Card Payment", "Hide from Budgets & Trends", "Investments", "Kids", "Savings", "Transfer", "Transfer for Cash Spending", "Transfer to Savings"],
  "Income": ["Bonus", "Income", "Interest Income", "Paycheck", "Rental"]
}

In [ ]:
mc_reverse = {v:k for k in mc for v in mc[k]}
len(mc_reverse)

In [ ]:
df = pd.read_csv('../data/transactions.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].apply(lambda x: x.year)
df['month'] = df['Date'].apply(lambda x: x.month)
df['day'] = df['Date'].apply(lambda x: x.day)
df['master_category'] = df['Category'].replace(mc_reverse)
df.columns = [c.replace(' ', '_').lower() for c in df.columns]
df.head()

In [ ]:
df.info()

# Create Database

* create raw transactions table
* create summary table
    * group to top-level category
    * aggregate by day
    * add MA's

In [ ]:
# create transactions table, replace if exists
conn = sqlite3.connect('../data/transactions.db')
df.to_sql('transactions', conn, index=None, if_exists='replace')
conn.close()

In [ ]:
conn = sqlite3.connect('../data/transactions.db')
loaded = pd.read_sql("""
    SELECT date, amount
    FROM transactions
    WHERE transaction_type == 'credit' AND category <> 'Transfer'
""", conn)
conn.close()
loaded.head()

# Aggregate

In [ ]:
daily = df.copy()
daily = daily[['date', 'amount', 'master_category']].groupby(['date', 'master_category']).sum()
daily.head(20)

In [ ]:
monthly = df.copy().set_index('date')
monthly = monthly[['master_category', 'amount']].groupby([lambda x: pd.to_datetime(f'{x.year}-{x.month:02d}'), 'master_category']).sum()
monthly['isExpense'] = monthly.index.get_level_values('master_category') != 'Income'
monthly['io'] = 'Income'
monthly.loc['isExpense', 'io'] = 'Expense'
monthly.pivot(columns='isExpense')
#monthly = monthly.pivot(columns=monthly[monthly.index.get_level_values('master_category') == 'Income'], values='amount')

monthly.head(20)

In [ ]:
monthly['isExpense']

## Timed aggregation

In [ ]:
monthly.head()